# Preparazione dati per Power BI

### Setup e Import

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import os

print("Implementazione per ecosistema startup italiane")

Implementazione per ecosistema startup italiane


### Caricamento Dati

In [3]:
# Caricamento dataset principali
startup_df = pd.read_csv(r'C:\Users\alexa\OneDrive\Documenti\Final Capstone with Epicode\VEM\startup_with_regions.csv', sep=';', encoding='utf-8', keep_default_na=False, na_values=[''])
vem_df = pd.read_csv(r'C:\Users\alexa\OneDrive\Documenti\Final Capstone with Epicode\VEM\VEM 2020 2024.csv', sep=';', encoding='utf-8')
pem_df = pd.read_csv(r'C:\Users\alexa\OneDrive\Documenti\Final Capstone with Epicode\VEM\CSV PEM\PEM 2020 - 2024.csv', sep=';', encoding='utf-8', keep_default_na=False, na_values=[''])

print(f"Dataset caricati:")
print(f"- Startup Registro Imprese: {len(startup_df)} records")
print(f"- VEM Investimenti: {len(vem_df)} records")
print(f"- PEM Investimenti: {len(pem_df)} records")

Dataset caricati:
- Startup Registro Imprese: 12292 records
- VEM Investimenti: 1364 records
- PEM Investimenti: 1906 records


### Pulizia Colonne

In [4]:
# Pulizia nomi colonne
startup_df.columns = startup_df.columns.str.strip()
vem_df.columns = vem_df.columns.str.strip()
pem_df.columns = pem_df.columns.str.strip()

print("Colonne pulite per tutti i dataset")

Colonne pulite per tutti i dataset


### Dimensiuni Condivise

In [5]:
# Dim_Date (2013-2030)
date_range = pd.date_range(start='2013-01-01', end='2030-12-31', freq='D')
dim_date = pd.DataFrame({
    'FullDate': date_range,
    'Year': date_range.year,
    'Quarter': date_range.quarter,
    'Month': date_range.month,
    'MonthName': date_range.month_name(),
    'IsWeekend': date_range.weekday >= 5
})

print(f"Dim_Date creata: {len(dim_date)} records")

# Dim_Geography
regioni_startup = set(startup_df['regione'].dropna().unique())
regioni_vem = set(vem_df['Regione'].dropna().unique())
regioni_pem = set(pem_df['Regione'].dropna().unique())

# Unisci tutte le regioni
all_regions = regioni_startup.union(regioni_vem).union(regioni_pem)
all_regions = sorted([r for r in all_regions if r and r != ''])

dim_geography = pd.DataFrame({
    'RegionKey': range(1, len(all_regions) + 1),
    'RegionName': all_regions
})

# Aggiungi macro regioni
macro_regions = {
    'LOMBARDIA': 'Nord', 
    'PIEMONTE': 'Nord', 
    'VENETO': 'Nord', 
    'LIGURIA': 'Nord',
    'EMILIA-ROMAGNA': 'Nord', 
    'TRENTINO-ALTO ADIGE': 'Nord', 
    'FRIULI-VENEZIA GIULIA': 'Nord',
    "VALLE D'AOSTA": 'Nord', 
    'TOSCANA': 'Centro', 
    'LAZIO': 'Centro', 
    'UMBRIA': 'Centro',
    'MARCHE': 'Centro', 
    'ABRUZZO': 'Centro', 
    'CAMPANIA': 'Sud', 
    'CALABRIA': 'Sud',
    'PUGLIA': 'Sud', 
    'BASILICATA': 'Sud', 
    'SICILIA': 'Sud', 
    'SARDEGNA': 'Sud', 
    'MOLISE': 'Sud'
}

dim_geography['MacroRegion'] = dim_geography['RegionName'].map(macro_regions)
dim_geography['MacroRegion'] = dim_geography['MacroRegion'].fillna('Altro')

print(f"Dim_Geography creata: {len(dim_geography)} records")

# Crea mapping per Region Key
region_mapping = dict(zip(dim_geography['RegionName'], dim_geography['RegionKey']))

Dim_Date creata: 6574 records
Dim_Geography creata: 22 records


### DIM_Sector

In [6]:
# Raccogli settori da tutti i dataset
settori_startup = set(startup_df['settore'].dropna().unique())
settori_vem = set(vem_df['Settore'].dropna().unique())
settori_pem = set(pem_df['Sector 1° level'].dropna().unique())

all_sectors = settori_startup.union(settori_vem).union(settori_pem)
all_sectors = sorted([s for s in all_sectors if s and s != ''])

dim_sector = pd.DataFrame({
    'SectorKey': range(1, len(all_sectors) + 1),
    'SectorName': all_sectors
})

# Classifica settori
dim_sector['SectorCategory'] = dim_sector['SectorName'].apply(lambda x: 
    'ICT' if 'ICT' in str(x).upper() or 'TECH' in str(x).upper() else
    'SERVIZI' if 'SERVIZI' in str(x).upper() else
    'INDUSTRIA' if 'INDUSTRIA' in str(x).upper() else
    'ALTRO'
)

print(f"Dim_Sector creata: {len(dim_sector)} records")

sector_mapping = dict(zip(dim_sector['SectorName'], dim_sector['SectorKey']))


Dim_Sector creata: 40 records


#### Dim_Investor

In [7]:
# Raccogli investitori da VEM e PEM
investitori_vem = set(vem_df['Lead Investor'].dropna().unique())
investitori_pem = set(pem_df['Lead Investor/Advisor'].dropna().unique())

all_investors = investitori_vem.union(investitori_pem)
all_investors = sorted([i for i in all_investors if i and i != ''])

dim_investor = pd.DataFrame({
    'InvestorKey': range(1, len(all_investors) + 1),
    'InvestorName': all_investors
})

# Classifica investitori (logica semplificata)
dim_investor['InvestorType'] = dim_investor['InvestorName'].apply(lambda x:
    'VC' if any(word in str(x).upper() for word in ['VENTURE', 'VENTURES', 'VC']) else
    'PE' if any(word in str(x).upper() for word in ['CAPITAL', 'EQUITY', 'FUND']) else
    'CORPORATE' if any(word in str(x).upper() for word in ['SPA', 'SRL', 'GROUP']) else
    'GOVERNMENT' if any(word in str(x).upper() for word in ['INVITALIA', 'CDP', 'STATO']) else
    'OTHER'
)

print(f"Dim_Investor creata: {len(dim_investor)} records")

investor_mapping = dict(zip(dim_investor['InvestorName'], dim_investor['InvestorKey']))

Dim_Investor creata: 1122 records


### Funzioni di utilita

In [8]:
def parse_date_to_key(date_str):
    """Converte data string in DateKey"""
    if pd.isna(date_str):
        return None
    
    try:
        if isinstance(date_str, str):
            for fmt in ['%d/%m/%Y', '%Y-%m-%d', '%d-%m-%Y']:
                try:
                    date_obj = datetime.strptime(date_str, fmt)
                    return int(date_obj.strftime('%Y%m%d'))
                except:
                    continue
        return None
    except:
        return None

def get_region_key(region_name):
    """Ottiene RegionKey da nome regione"""
    return region_mapping.get(region_name, 0)

def get_sector_key(sector_name):
    """Ottiene SectorKey da nome settore"""
    return sector_mapping.get(sector_name, 0)

def get_investor_key(investor_name):
    """Ottiene InvestorKey da nome investitore"""
    return investor_mapping.get(investor_name, 0)

print("Funzioni di utilità create")


Funzioni di utilità create


### Fact Startup Registry

In [9]:
fact_startup_registry = pd.DataFrame({
    'StartupID': range(1, len(startup_df) + 1),
    'RegionKey': startup_df['regione'].apply(get_region_key),
    'SectorKey': startup_df['settore'].apply(get_sector_key),
    'Data_Iscrizione_StartUp': startup_df['data iscrizione alla sezione delle startup'],
    'DateKey_InizioAttivita': startup_df['data inizio dell\'esercizio effettivo dell\'attività'].apply(parse_date_to_key),
    'Denominazione': startup_df['denominazione'],
    'nome': startup_df['nome'],
    'Natura giuridica': startup_df['nat.giuridica'],
    'Provincia': startup_df['pv'],
    'Comune': startup_df['comune'],
    'Regione' : startup_df['regione'],
    'AtecoCode': startup_df['ateco 2007'],
    'Data iscrizione al Registro Imprese': startup_df['data iscrizione al Registro Imprese'],
    'Settore': startup_df['settore'],
    'AltoValoreTecnologico': startup_df['alto valore tecnologico'],
    'VocazioneSociale': startup_df['vocazione sociale'],
    'Classe addetti' : startup_df['classe addetti'],
    'Calsse val. prodozione' : startup_df['classe val.prod.'],
    'ClasseCapitaleSociale': startup_df['classe cap.sociale'],
    'SpeseRnD': startup_df['spese in ricerca e sviluppo'],
    'ForzaLavoroTitoli': startup_df['forza lavoro con titoli'],
    'PossessoBrevetti': startup_df['possesso di brevetti'],
    'PrevalenzaFemminile': startup_df['prevalenza femminile'],
    'PrevalenzaGiovanile': startup_df['prevalenza giovanile'],
    'PrevalenzaStraniera': startup_df['prevalenza straniera'],
    'Status': startup_df['status'],
    'TipoSocieta': startup_df['tipo_societa'],
    'Year': startup_df['Year'],
    'NumeroStartup': 1  # Measure
})

print(f"Fact_Startup_Registry creata: {len(fact_startup_registry)} records")


Fact_Startup_Registry creata: 12292 records


### Fact_VEM_Investments

In [10]:
fact_vem_investments = pd.DataFrame({
    'VEMDealID': range(1, len(vem_df) + 1),
    'RegionKey': vem_df['Regione'].apply(get_region_key),
    'SectorKey': vem_df['Settore'].apply(get_sector_key),
    'InvestorKey': vem_df['Lead Investor'].apply(get_investor_key),
    'TargetCompany': vem_df['Target Company'],
    'LeadInvestor': vem_df['Lead Investor'],
    'CoInvestors': vem_df['Co-Investors'],
    'InvestmentStage': vem_df['Investment\nStage'],
    'DealOrigination': vem_df['Deal\nOrigination'],
    'Regione': vem_df['Regione'],
    'Country' : vem_df['Country'],
    'AnnoFondazione': vem_df['Anno di\nFondazione'],
    'Settore': vem_df['Settore'],
    'DescrizioneAttivita': vem_df['Descrizione Attività'],
    'AnnoReportVEM': vem_df['Anno report VEM'],
    'AmountInvested_EUR_MLN': vem_df['Ammontare\nInvestito Totale\n(Euro Mln)'],
    'NumeroInvestimenti': 1  # Measure
})

# Aggiungi colonne calcolate
fact_vem_investments['EtaAlInvestimento'] = fact_vem_investments['AnnoReportVEM'] - fact_vem_investments['AnnoFondazione']
fact_vem_investments['IsRealStartup'] = fact_vem_investments['EtaAlInvestimento'] <= 5

print(f"Fact_VEM_Investments creata: {len(fact_vem_investments)} records")


Fact_VEM_Investments creata: 1364 records


### Fact_PEM_Investments

In [11]:
fact_pem_investments = pd.DataFrame({
    'PEMDealID': range(1, len(pem_df) + 1),
    'RegionKey': pem_df['Regione'].apply(get_region_key),
    'SectorKey': pem_df['Sector 1° level'].apply(get_sector_key),
    'InvestorKey': pem_df['Lead Investor/Advisor'].apply(get_investor_key),
    'TargetCompany': pem_df['Target company'],
    'LeadInvestor': pem_df['Lead Investor/Advisor'],
    'CoInvestors': pem_df['Co-investors'],
    'InvestmentStage': pem_df['Investment stage'],
    'DealOrigination': pem_df['Deal origination'],
    'Regione': pem_df['Regione'],
    '2.Regione' : pem_df['2 Region'],
    'Sector1Level': pem_df['Sector 1° level'],
    'SectorSIC': pem_df['Sector\n(SIC Code)'],
    'AnnoReportPEM': pem_df['ANNO REPORT PEM'],
    'PlatformCompany': pem_df['Platform company'],
    'AmountInvested_EUR_MLN': pem_df['Invested\namount\n(€ mln)'],
    'AcquiredStake_Percent': pem_df['Acquired\nstake\n(%)'],
    'NumeroInvestimenti': 1  # Measure
})

# Aggiungi colonne calcolate
fact_pem_investments['IsBuyout'] = fact_pem_investments['InvestmentStage'] == 'Buy Out'
fact_pem_investments['IsInfrastructure'] = fact_pem_investments['InvestmentStage'] == 'Infrastructure'

print(f"Fact_PEM_Investments creata: {len(fact_pem_investments)} records")

Fact_PEM_Investments creata: 1906 records


### Validazione Dati

In [12]:
print("\n--- VALIDAZIONE QUALITÀ DATI ---")

# Startup Registry
startup_completeness = {
    'Regione': fact_startup_registry['RegionKey'].notna().sum() / len(fact_startup_registry),
    'Settore': fact_startup_registry['SectorKey'].notna().sum() / len(fact_startup_registry),
    'Data Iscrizione': fact_startup_registry['Data_Iscrizione_StartUp'].notna().sum() / len(fact_startup_registry)
}

print("Startup Registry - Completezza dati:")
for campo, percentuale in startup_completeness.items():
    print(f"  {campo}: {percentuale:.1%}")

# VEM Investments
vem_completeness = {
    'Regione': fact_vem_investments['RegionKey'].notna().sum() / len(fact_vem_investments),
    'Settore': fact_vem_investments['SectorKey'].notna().sum() / len(fact_vem_investments),
    'Investitore': fact_vem_investments['InvestorKey'].notna().sum() / len(fact_vem_investments),
    'Importo': fact_vem_investments['AmountInvested_EUR_MLN'].notna().sum() / len(fact_vem_investments)
}

print("\nVEM Investments - Completezza dati:")
for campo, percentuale in vem_completeness.items():
    print(f"  {campo}: {percentuale:.1%}")

# PEM Investments
pem_completeness = {
    'Regione': fact_pem_investments['RegionKey'].notna().sum() / len(fact_pem_investments),
    'Settore': fact_pem_investments['SectorKey'].notna().sum() / len(fact_pem_investments),
    'Investitore': fact_pem_investments['InvestorKey'].notna().sum() / len(fact_pem_investments),
    'Importo': fact_pem_investments['AmountInvested_EUR_MLN'].notna().sum() / len(fact_pem_investments)
}

print("\nPEM Investments - Completezza dati:")
for campo, percentuale in pem_completeness.items():
    print(f"  {campo}: {percentuale:.1%}")



--- VALIDAZIONE QUALITÀ DATI ---
Startup Registry - Completezza dati:
  Regione: 100.0%
  Settore: 100.0%
  Data Iscrizione: 100.0%

VEM Investments - Completezza dati:
  Regione: 100.0%
  Settore: 100.0%
  Investitore: 100.0%
  Importo: 88.8%

PEM Investments - Completezza dati:
  Regione: 100.0%
  Settore: 100.0%
  Investitore: 100.0%
  Importo: 10.0%


### Statistiche Descrittive

In [13]:
print(fact_vem_investments['AmountInvested_EUR_MLN'].dtype)
print(fact_vem_investments['AmountInvested_EUR_MLN'].head(10))


object
0    0,53
1     1,7
2     0,7
3     1,2
4       3
5     NaN
6     0,2
7     2,3
8     1,7
9     NaN
Name: AmountInvested_EUR_MLN, dtype: object


In [14]:
fact_vem_investments['AmountInvested_EUR_MLN'] = (
    fact_vem_investments['AmountInvested_EUR_MLN']
    .astype(str)
    .str.replace(',', '.', regex=False)             # 1,5 → 1.5
    .str.replace(r'[^\d\.]', '', regex=True)        # Rimuove caratteri non numerici
    .replace(['', 'nan', 'NaN', 'None'], '0')       # Gestisce stringhe vuote o null
    .astype(float)                                  # Converte in float
)


In [15]:
print("\n--- STATISTICHE DESCRITTIVE ---")

# VEM - Startup vere vs mature
vem_real_startups = fact_vem_investments[fact_vem_investments['IsRealStartup'] == True]
vem_mature_companies = fact_vem_investments[fact_vem_investments['IsRealStartup'] == False]

print(f"VEM - Startup vere (≤5 anni): {len(vem_real_startups)} ({len(vem_real_startups)/len(fact_vem_investments):.1%})")
print(f"VEM - Aziende mature (>5 anni): {len(vem_mature_companies)} ({len(vem_mature_companies)/len(fact_vem_investments):.1%})")

# Investimenti per categoria
vem_startup_investments = vem_real_startups['AmountInvested_EUR_MLN'].sum()
vem_mature_investments = vem_mature_companies['AmountInvested_EUR_MLN'].sum()

print(f"\nInvestimenti VEM:")
print(f"  Startup vere: {vem_startup_investments:.1f}M € ({vem_startup_investments/(vem_startup_investments+vem_mature_investments):.1%})")
print(f"  Aziende mature: {vem_mature_investments:.1f}M € ({vem_mature_investments/(vem_startup_investments+vem_mature_investments):.1%})")

# PEM - Distribuzione per tipo investimento
pem_buyout = fact_pem_investments[fact_pem_investments['IsBuyout'] == True]
pem_other = fact_pem_investments[fact_pem_investments['IsBuyout'] == False]

print(f"\nPEM - Buyout: {len(pem_buyout)} ({len(pem_buyout)/len(fact_pem_investments):.1%})")
print(f"PEM - Altri: {len(pem_other)} ({len(pem_other)/len(fact_pem_investments):.1%})")




--- STATISTICHE DESCRITTIVE ---
VEM - Startup vere (≤5 anni): 928 (68.0%)
VEM - Aziende mature (>5 anni): 436 (32.0%)

Investimenti VEM:
  Startup vere: 3795.2M € (49.1%)
  Aziende mature: 3936.3M € (50.9%)

PEM - Buyout: 1491 (78.2%)
PEM - Altri: 415 (21.8%)


### Export per Power BI

In [16]:
# Esporta tutte le tabelle
tables_to_export = {
    'Dim_Date': dim_date,
    'Dim_Geography': dim_geography,
    'Dim_Sector': dim_sector,
    'Dim_Investor': dim_investor,
    'Fact_Startup_Registry': fact_startup_registry,
    'Fact_VEM_Investments': fact_vem_investments,
    'Fact_PEM_Investments': fact_pem_investments
}

print("\n=== EXPORT PER POWER BI ===")
for table_name, df in tables_to_export.items():
    filename = f'{table_name}.csv'
    df.to_csv(filename, index=False)
    print(f"✓ {filename} - {len(df)} records")

print("\nTutte le tabelle sono state esportate!")
print("Importa in Power BI e crea le relazioni tra le dimensioni e le fact table.")


=== EXPORT PER POWER BI ===
✓ Dim_Date.csv - 6574 records
✓ Dim_Geography.csv - 22 records
✓ Dim_Sector.csv - 40 records
✓ Dim_Investor.csv - 1122 records
✓ Fact_Startup_Registry.csv - 12292 records
✓ Fact_VEM_Investments.csv - 1364 records
✓ Fact_PEM_Investments.csv - 1906 records

Tutte le tabelle sono state esportate!
Importa in Power BI e crea le relazioni tra le dimensioni e le fact table.


In [17]:
import os
print(f"📂 Salvataggio nella cartella: {os.getcwd()}")


📂 Salvataggio nella cartella: C:\Users\alexa\OneDrive\Documenti\Final Capstone with Epicode\VEM\CSV PEM


### Relazioni Power BI

In [18]:
print("\n--- RELAZIONI DA CREARE IN POWER BI ---")
print("1. Dim_Date[DateKey] → Fact_Startup_Registry[DateKey_Iscrizione] (Many-to-One)")
print("2. Dim_Date[DateKey] → Fact_Startup_Registry[DateKey_InizioAttivita] (Many-to-One)")
print("3. Dim_Geography[RegionKey] → Fact_Startup_Registry[RegionKey] (Many-to-One)")
print("4. Dim_Geography[RegionKey] → Fact_VEM_Investments[RegionKey] (Many-to-One)")
print("5. Dim_Geography[RegionKey] → Fact_PEM_Investments[RegionKey] (Many-to-One)")
print("6. Dim_Sector[SectorKey] → Fact_Startup_Registry[SectorKey] (Many-to-One)")
print("7. Dim_Sector[SectorKey] → Fact_VEM_Investments[SectorKey] (Many-to-One)")
print("8. Dim_Sector[SectorKey] → Fact_PEM_Investments[SectorKey] (Many-to-One)")
print("9. Dim_Investor[InvestorKey] → Fact_VEM_Investments[InvestorKey] (Many-to-One)")
print("10. Dim_Investor[InvestorKey] → Fact_PEM_Investments[InvestorKey] (Many-to-One)")

print("\n=== IMPLEMENTAZIONE COMPLETATA ===")
print("Approccio metodologicamente corretto implementato!")


--- RELAZIONI DA CREARE IN POWER BI ---
1. Dim_Date[DateKey] → Fact_Startup_Registry[DateKey_Iscrizione] (Many-to-One)
2. Dim_Date[DateKey] → Fact_Startup_Registry[DateKey_InizioAttivita] (Many-to-One)
3. Dim_Geography[RegionKey] → Fact_Startup_Registry[RegionKey] (Many-to-One)
4. Dim_Geography[RegionKey] → Fact_VEM_Investments[RegionKey] (Many-to-One)
5. Dim_Geography[RegionKey] → Fact_PEM_Investments[RegionKey] (Many-to-One)
6. Dim_Sector[SectorKey] → Fact_Startup_Registry[SectorKey] (Many-to-One)
7. Dim_Sector[SectorKey] → Fact_VEM_Investments[SectorKey] (Many-to-One)
8. Dim_Sector[SectorKey] → Fact_PEM_Investments[SectorKey] (Many-to-One)
9. Dim_Investor[InvestorKey] → Fact_VEM_Investments[InvestorKey] (Many-to-One)
10. Dim_Investor[InvestorKey] → Fact_PEM_Investments[InvestorKey] (Many-to-One)

=== IMPLEMENTAZIONE COMPLETATA ===
Approccio metodologicamente corretto implementato!
